In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [2]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.4,
        horizontal_flip=True,
        fill_mode='nearest')


In [3]:
import os 
print(os.getcwd()) ##Previous Directory
os.chdir(r"/Users/audreatabadero/Downloads/finaltry1/MS7") ##Change with your current working directory #change din i2
print(os.getcwd())  ##Current Working Directory

#removes hidden files
root = "/Users/audreatabadero/Downloads/finaltry1"
for item in os.listdir(root):
    if not item.startswith('.') and os.path.isfile(os.path.join(root, item)):
        print (item)


/Users/audreatabadero/Desktop/tensor123
/Users/audreatabadero/Downloads/finaltry1/MS7


In [6]:
for path in os.listdir():
    print(path)
    img = load_img(f"/Users/audreatabadero/Downloads/finaltry1/MS7/{path}") #change i2
    x = img_to_array(img)    # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)
    i = 0
    for batch in datagen.flow(x, batch_size=1, # vvv change directory din here
                          save_to_dir="/Users/audreatabadero/Downloads/finaltry1/train/MS7", save_prefix='img', save_format='jpeg'):
        i += 1
        if i > 10:     ## creates 10 image from 1 image 
            break  

v005.jpg
v011.jpg


KeyboardInterrupt: 

In [4]:
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model 
from keras.applications.vgg16 import VGG16 
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np

In [5]:
### Defining Image size
IMAGE_SIZE = [224, 224]

### Loading model
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
### Freezing layers
for layer in vgg.layers:  
  layer.trainable = False

### adding a 3 node final layer for predicion
x = Flatten()(vgg.output)
prediction = Dense(7, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
### Generating Summary
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
model.compile( loss='categorical_crossentropy',  
               optimizer='adam',  
               metrics=['accuracy'])

In [10]:

# train_datagen = ImageDataGenerator(rescale = 1./255,                          
#                                     shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                     horizontal_flip= True)
# train_datagen = train_datagen.flow_from_directory('/Users/audreatabadero/Downloads/finaltry1/train', target_size=(224,224), batch_size=2001, class_mode="categorical")
# X, y = train_datagen.next()


# training_set = train_datagen.flow_from_directory('/Users/audreatabadero/Downloads/finaltry1/train',
#                                            target_size = (224, 224),
#                                              batch_size = 16,                             
#                                         class_mode = 'categorical')

train_datagen = ImageDataGenerator(rescale=1. / 255)
itr = train_datagen.flow_from_directory(
"/Users/audreatabadero/Downloads/finaltry1/train",
target_size=(224, 224),
batch_size=16,
class_mode='categorical')

X, y = itr.next()

Found 2001 images belonging to 7 classes.


In [11]:
test_datagen = ImageDataGenerator(rescale = 1./255)
# test_set = test_datagen.flow_from_directory('/Users/audreatabadero/Downloads/finaltry1/validation',
#                                          target_size = (224, 224),
#                                          batch_size=16,
#                                          class_mode = 'categorical')

test_datagen = test_datagen.flow_from_directory('/Users/audreatabadero/Downloads/finaltry1/validation', target_size=(224,224), batch_size=16, class_mode="categorical")

Found 185 images belonging to 7 classes.


In [9]:




r = model.fit(X,y, batch_size = 16, epochs=2, validation_data=test_datagen)
# r = model.fit(X,y, batch_size = 16, epochs=5, validation_split=0.3)

# r = model.fit_generator(training_set,  validation_data=test_set,  epochs=25,steps_per_epoch=len(training_set),validation_steps=len(test_set))

Epoch 1/2
1/1 [==============================] - 52s 52s/step - loss: 2.0588 - accuracy: 0.1250 - val_loss: 4.0286 - val_accuracy: 0.3135
Epoch 2/2
1/1 [==============================] - 47s 47s/step - loss: 1.5776 - accuracy: 0.5000 - val_loss: 5.1593 - val_accuracy: 0.1189


In [80]:
model.save("ripeness.h5")